In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GaussianNoise, BatchNormalization
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import math

# # Load the data
# meth = pd.read_csv('HNSC_methy.csv')



# Load the data with the correct delimiter ('\t' for tab-separated values)
meth = pd.read_csv('/content/HNSC_methy (1).csv', sep='\t')

# Verify the data
print(meth.head())

# Ensure only numeric columns are processed
meth_numeric = meth.select_dtypes(include=['float64', 'int64'])

scaler = StandardScaler()
meth_scaled = scaler.fit_transform(meth)

# Define the autoencoder model
def create_denoising_autoencoder(input_dim, regularization=1e-5, dropout_rate=0.2, noise_factor=0.3):
    input_layer = Input(shape=(input_dim,))

    # Add Gaussian Noise for denoising
    noisy_input = GaussianNoise(noise_factor)(input_layer)

    # Encoder
    encoded = Dense(512, activation='relu', kernel_regularizer=l2(regularization))(noisy_input)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    encoded = Dense(256, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    encoded = Dense(128, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    encoded = Dense(64, activation='relu', activity_regularizer=l1(1e-5))(encoded)  # Sparse representation

    # Decoder
    decoded = Dense(128, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dropout(dropout_rate)(decoded)
    decoded = Dense(256, activation='relu', kernel_regularizer=l2(regularization))(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dropout(dropout_rate)(decoded)
    decoded = Dense(input_dim, activation='linear')(decoded)

    autoencoder = Model(input_layer, decoded)
    return autoencoder

# Define the autoencoder model
input_dim = meth_scaled.shape[1]
autoencoder = create_denoising_autoencoder(input_dim)

# Compile the autoencoder model with MSLE loss
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredLogarithmicError())

# Define the cosine annealing learning rate scheduler
def cosine_annealing(epoch, lr, initial_lr=0.0001, min_lr=1e-5, T=2000):
    """Cosine annealing learning rate schedule."""
    cos_inner = (math.pi * (epoch % T)) / T
    return min_lr + (initial_lr - min_lr) * (1 + math.cos(cos_inner)) / 2

lr_schedule = LearningRateScheduler(lambda epoch, lr: cosine_annealing(epoch, lr, T=2000))

# Train the autoencoder on the full dataset
history = autoencoder.fit(
    meth_scaled,
    meth_scaled,
    epochs=2000,
    batch_size=128,
    callbacks=[lr_schedule]
)

# Print the final training loss
train_loss = history.history['loss'][-1]
print(f'Training loss: {train_loss}')


In [ ]:
!pip install seaborn
!pip install matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Get the reconstructed data from the autoencoder
X_reconstructed = autoencoder.predict(meth_scaled)

# Calculate reconstruction errors (absolute difference between original and reconstructed data)
reconstruction_errors = np.abs(meth_scaled - X_reconstructed)

# Define a function to plot the combined heatmaps
def plot_combined_heatmaps(original, reconstructed, errors, title):
    # Create a figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))

    # Plot the heatmap for the original data
    sns.heatmap(original[:50], ax=axes[0], cmap='viridis')
    axes[0].set_title('Original Data')

    # Plot the heatmap for the reconstructed data
    sns.heatmap(reconstructed[:50], ax=axes[1], cmap='viridis')
    axes[1].set_title('Reconstructed Data')

    # Plot the heatmap for the reconstruction error
    sns.heatmap(errors[:50], ax=axes[2], cmap='Reds')
    axes[2].set_title('Reconstruction Error')

    # Set the main title for the entire plot
    plt.suptitle(title)

    # Display the plot
    plt.show()

# Plot the heatmaps for the first 50 samples of each dataset
plot_combined_heatmaps(meth_scaled, X_reconstructed, reconstruction_errors, 'Reconstruction Errors of the Autoencoder')


In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):

    plt.plot(history.history['loss'], label='Training Loss')

    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()


plot_loss(history)


In [ ]:
# Save reconstructed scaled data
reconstructed_df_scaled = pd.DataFrame(X_reconstructed, columns=meth.columns)
reconstructed_df_scaled.to_csv('HNSC_Reconstructed_meth.csv', index=False)

# Optionally inverse-transform and save
X_reconstructed_original = scaler.inverse_transform(X_reconstructed)
reconstructed_df_original = pd.DataFrame(X_reconstructed_original, columns=meth.columns)
reconstructed_df_original.to_csv('HNSC_Reconstruction_original.csv', index=False)

print("Reconstructed datasets saved to 'reconstructed_meth_scaled.csv' and 'reconstructed_meth_original.csv'.")


In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load data
df = pd.read_csv('/content/HNSC_Reconstruction_original.csv')  # Rows: samples, Columns: features (no true labels)
X = df.values

# Step 2: Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
# Step 4: Choose number of clusters
chosen_k = 2  # adjust based on elbow plot

# Step 5: Initial clustering to get pseudo-labels
initial_kmeans = KMeans(n_clusters=chosen_k, random_state=42)
pseudo_labels = initial_kmeans.fit_predict(X_scaled)

# Step 6: Apply SMOTE on pseudo-labels
smote = SMOTE(random_state=42)
X_smote, pseudo_labels_smote = smote.fit_resample(X_scaled, pseudo_labels)

# Step 7: Final clustering on SMOTE data
kmeans_final = KMeans(n_clusters=chosen_k, random_state=42)
final_clusters = kmeans_final.fit_predict(X_smote)

# Step 8: Create DataFrame with biological feature names and cluster labels
result_df = pd.DataFrame(X_smote, columns=df.columns)  # preserve real feature names here
result_df['Cluster'] = final_clusters

print(result_df.head())
print("\nCluster Counts:\n", result_df['Cluster'].value_counts())

# Step 9: t-SNE visualization of clusters
tsne = TSNE(n_components=2, random_state=42)
X_embedded = tsne.fit_transform(result_df.drop('Cluster', axis=1))

plt.figure(figsize=(8, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=result_df['Cluster'], cmap='viridis', s=10)
plt.title("t-SNE Visualization of Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label='Cluster')
plt.show()

# Step 10: Internal cluster validation scores
silhouette = silhouette_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
db_score = davies_bouldin_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
ch_score = calinski_harabasz_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])

print("\nInternal Validation Metrics:")
print("Silhouette Score:", round(silhouette, 4))
print("Davies-Bouldin Score:", round(db_score, 4))
print("Calinski-Harabasz Score:", round(ch_score, 4))

# Step 11: Biomarker discovery (differential features across clusters)
X_biomarker = result_df.drop('Cluster', axis=1).values
y_clusters = result_df['Cluster'].values

anova_pvals = []
kruskal_pvals = []

for i in range(X_biomarker.shape[1]):
    groups = [X_biomarker[y_clusters == c, i] for c in np.unique(y_clusters)]

    # ANOVA
    try:
        _, p_anova = f_oneway(*groups)
    except:
        p_anova = 1.0
    anova_pvals.append(p_anova)

    # Kruskal-Wallis
    try:
        _, p_kruskal = kruskal(*groups)
    except:
        p_kruskal = 1.0
    kruskal_pvals.append(p_kruskal)

biomarker_df = pd.DataFrame({
    'Feature': df.columns,  # use original biological names here
    'ANOVA_pval': anova_pvals,
    'Kruskal_pval': kruskal_pvals
})

biomarker_df.sort_values('Kruskal_pval', inplace=True)

print("\nTop Biomarker Candidates (based on Kruskal-Wallis test):")
print(biomarker_df.head(20))

# Step 12: Print only top biomarker names
top_n = 20
top_biomarkers = biomarker_df.head(top_n)['Feature'].values

print("\nTop Biomarker Feature Names:")
for name in top_biomarkers:
    print(name)

# Step 13: Heatmap of top biomarkers across samples sorted by cluster
top_features = biomarker_df.head(top_n)['Feature'].values
heatmap_data = result_df[top_features]
heatmap_data['Cluster'] = result_df['Cluster'].values
heatmap_data_sorted = heatmap_data.sort_values('Cluster')
cluster_labels = heatmap_data_sorted['Cluster']
heatmap_data_sorted = heatmap_data_sorted.drop('Cluster', axis=1)

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data_sorted.T, cmap='viridis', cbar=True, xticklabels=False)
plt.title("Heatmap of Top Biomarker Features Across Clusters")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load data
df = pd.read_csv('/content/HNSC_methy (1).csv', sep='\t')  # Rows: samples, Columns: features (no true labels)
X = df.values

# Step 2: Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X)
    inertia.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
# Step 4: Choose number of clusters
chosen_k = 2  # adjust based on elbow plot

# Step 5: Initial clustering to get pseudo-labels
initial_kmeans = KMeans(n_clusters=chosen_k, random_state=42)
pseudo_labels = initial_kmeans.fit_predict(X_scaled)

# Step 6: Apply SMOTE on pseudo-labels
smote = SMOTE(random_state=42)
X_smote, pseudo_labels_smote = smote.fit_resample(X_scaled, pseudo_labels)

# Step 7: Final clustering on SMOTE data
kmeans_final = KMeans(n_clusters=chosen_k, random_state=42)
final_clusters = kmeans_final.fit_predict(X_smote)

# Step 8: Create DataFrame with biological feature names and cluster labels
result_df = pd.DataFrame(X_smote, columns=df.columns)  # preserve real feature names here
result_df['Cluster'] = final_clusters

print(result_df.head())
print("\nCluster Counts:\n", result_df['Cluster'].value_counts())

# Step 9: t-SNE visualization of clusters
tsne = TSNE(n_components=2, random_state=42)
X_embedded = tsne.fit_transform(result_df.drop('Cluster', axis=1))

plt.figure(figsize=(8, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=result_df['Cluster'], cmap='viridis', s=10)
plt.title("t-SNE Visualization of Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label='Cluster')
plt.show()

# Step 10: Internal cluster validation scores
silhouette = silhouette_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
db_score = davies_bouldin_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
ch_score = calinski_harabasz_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])

print("\nInternal Validation Metrics:")
print("Silhouette Score:", round(silhouette, 4))
print("Davies-Bouldin Score:", round(db_score, 4))
print("Calinski-Harabasz Score:", round(ch_score, 4))

# Step 11: Biomarker discovery (differential features across clusters)
X_biomarker = result_df.drop('Cluster', axis=1).values
y_clusters = result_df['Cluster'].values

anova_pvals = []
kruskal_pvals = []

for i in range(X_biomarker.shape[1]):
    groups = [X_biomarker[y_clusters == c, i] for c in np.unique(y_clusters)]

    # ANOVA
    try:
        _, p_anova = f_oneway(*groups)
    except:
        p_anova = 1.0
    anova_pvals.append(p_anova)

    # Kruskal-Wallis
    try:
        _, p_kruskal = kruskal(*groups)
    except:
        p_kruskal = 1.0
    kruskal_pvals.append(p_kruskal)

biomarker_df = pd.DataFrame({
    'Feature': df.columns,  # use original biological names here
    'ANOVA_pval': anova_pvals,
    'Kruskal_pval': kruskal_pvals
})

biomarker_df.sort_values('Kruskal_pval', inplace=True)

print("\nTop Biomarker Candidates (based on Kruskal-Wallis test):")
print(biomarker_df.head(20))

# Step 12: Print only top biomarker names
top_n = 20
top_biomarkers = biomarker_df.head(top_n)['Feature'].values

print("\nTop Biomarker Feature Names:")
for name in top_biomarkers:
    print(name)

# Step 13: Heatmap of top biomarkers across samples sorted by cluster
top_features = biomarker_df.head(top_n)['Feature'].values
heatmap_data = result_df[top_features]
heatmap_data['Cluster'] = result_df['Cluster'].values
heatmap_data_sorted = heatmap_data.sort_values('Cluster')
cluster_labels = heatmap_data_sorted['Cluster']
heatmap_data_sorted = heatmap_data_sorted.drop('Cluster', axis=1)

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data_sorted.T, cmap='viridis', cbar=True, xticklabels=False)
plt.title("Heatmap of Top Biomarker Features Across Clusters")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

mRNA

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GaussianNoise, BatchNormalization
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import math


# Load the data with the correct delimiter ('\t' for tab-separated values)
mRNA = pd.read_csv('/content/HNSC_mrna (2).csv', sep='\t')

# Verify the data
print(mRNA.head())

# Ensure only numeric columns are processed
mRNA_numeric = mRNA.select_dtypes(include=['float64', 'int64'])

scaler = StandardScaler()
mRNA_scaled = scaler.fit_transform(mRNA)


# Define the autoencoder model
def create_denoising_autoencoder(input_dim, regularization=1e-5, dropout_rate=0.2, noise_factor=0.3):
    input_layer = Input(shape=(input_dim,))

    # Add Gaussian Noise for denoising
    noisy_input = GaussianNoise(noise_factor)(input_layer)

    # Encoder
    encoded = Dense(512, activation='relu', kernel_regularizer=l2(regularization))(noisy_input)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    encoded = Dense(256, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    encoded = Dense(128, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    encoded = Dense(64, activation='relu', activity_regularizer=l1(1e-5))(encoded)  # Sparse representation

    # Decoder
    decoded = Dense(128, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dropout(dropout_rate)(decoded)
    decoded = Dense(256, activation='relu', kernel_regularizer=l2(regularization))(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dropout(dropout_rate)(decoded)
    decoded = Dense(input_dim, activation='linear')(decoded)

    autoencoder = Model(input_layer, decoded)
    return autoencoder

# Define the autoencoder model
input_dim = mRNA_scaled.shape[1]
autoencoder = create_denoising_autoencoder(input_dim)

# Compile the autoencoder model with MSLE loss
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredLogarithmicError())

# Define the cosine annealing learning rate scheduler
def cosine_annealing(epoch, lr, initial_lr=0.0001, min_lr=1e-5, T=2000):
    """Cosine annealing learning rate schedule."""
    cos_inner = (math.pi * (epoch % T)) / T
    return min_lr + (initial_lr - min_lr) * (1 + math.cos(cos_inner)) / 2

lr_schedule = LearningRateScheduler(lambda epoch, lr: cosine_annealing(epoch, lr, T=2000))

# Train the autoencoder on the full mRNA dataset
history_mRNA = autoencoder.fit(
    mRNA_scaled,
    mRNA_scaled,
    epochs=2000,
    batch_size=128,
    callbacks=[lr_schedule]
)

# Print the final training loss
train_loss_mRNA = history_mRNA.history['loss'][-1]
print(f'Training loss for mRNA dataset: {train_loss_mRNA}')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Reconstruct the data using the trained autoencoder
mRNA_reconstructed = autoencoder.predict(mRNA_scaled)

# Convert the reconstructed data back to the original scale
#mRNA_reconstructed_original = scaler.inverse_transform(mRNA_reconstructed)

# Calculate the reconstruction error (absolute error between original and reconstructed data)
reconstruction_error = np.abs(mRNA_scaled - mRNA_reconstructed)

def plot_combined_heatmaps(original, reconstructed, error, title):
    # Plot the first 50 samples for visualization (adjust as needed)
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))

    # Heatmap for Original Data
    sns.heatmap(original[:50], ax=axes[0], cmap='viridis', cbar_kws={'label': 'Value'})
    axes[0].set_title('Original mRNA Data')

    # Heatmap for Reconstructed Data
    sns.heatmap(reconstructed[:50], ax=axes[1], cmap='viridis', cbar_kws={'label': 'Value'})
    axes[1].set_title('Reconstructed mRNA Data')

    # Heatmap for Reconstruction Error
    sns.heatmap(error[:50], ax=axes[2], cmap='Reds', cbar_kws={'label': 'Reconstruction Error'})
    axes[2].set_title('Reconstruction Error')

    # Overall title and layout adjustments
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

# Plot heatmaps for original, reconstructed data, and reconstruction error
plot_combined_heatmaps(mRNA_scaled, mRNA_reconstructed, reconstruction_error, 'mRNA Data Reconstruction Heatmap')


In [ ]:
import matplotlib.pyplot as plt

def plot_loss_mRNA(history_mRNA):
    # Plot the training loss
    plt.plot(history_mRNA.history['loss'], label='Training Loss')

    plt.title('mRNA Autoencoder Training Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

# Plot the training loss for the mRNA dataset
plot_loss_mRNA(history_mRNA)


In [ ]:
# Reconstruct the mRNA data using the trained autoencoder
mRNA_reconstructed = autoencoder.predict(mRNA_scaled)

# Save reconstructed scaled data
reconstructed_mRNA_df_scaled = pd.DataFrame(mRNA_reconstructed, columns=mRNA.columns)
reconstructed_mRNA_df_scaled.to_csv('HNSC_mrna_reconstructed.csv', index=False)

# Optionally inverse-transform and save
mRNA_reconstructed_original = scaler.inverse_transform(mRNA_reconstructed)
reconstructed_mRNA_df_original = pd.DataFrame(mRNA_reconstructed_original, columns=mRNA.columns)
reconstructed_mRNA_df_original.to_csv('HNSC_mrna_original.csv', index=False)

print("Reconstructed mRNA datasets saved to 'reconstructed_mRNA_scaled.csv' and 'reconstructed_mRNA_original.csv'.")


In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load data
df = pd.read_csv('/content/HNSC_mrna_original.csv')  # Rows: samples, Columns: features (no true labels)
X = df.values

# Step 2: Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
# Step 4: Choose number of clusters
chosen_k = 2  # adjust based on elbow plot

# Step 5: Initial clustering to get pseudo-labels
initial_kmeans = KMeans(n_clusters=chosen_k, random_state=42)
pseudo_labels = initial_kmeans.fit_predict(X_scaled)

# Step 6: Apply SMOTE on pseudo-labels
smote = SMOTE(random_state=42)
X_smote, pseudo_labels_smote = smote.fit_resample(X_scaled, pseudo_labels)

# Step 7: Final clustering on SMOTE data
kmeans_final = KMeans(n_clusters=chosen_k, random_state=42)
final_clusters = kmeans_final.fit_predict(X_smote)

# Step 8: Create DataFrame with biological feature names and cluster labels
result_df = pd.DataFrame(X_smote, columns=df.columns)  # preserve real feature names here
result_df['Cluster'] = final_clusters

print(result_df.head())
print("\nCluster Counts:\n", result_df['Cluster'].value_counts())

# Step 9: t-SNE visualization of clusters
tsne = TSNE(n_components=2, random_state=42)
X_embedded = tsne.fit_transform(result_df.drop('Cluster', axis=1))

plt.figure(figsize=(8, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=result_df['Cluster'], cmap='viridis', s=10)
plt.title("t-SNE Visualization of Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label='Cluster')
plt.show()

# Step 10: Internal cluster validation scores
silhouette = silhouette_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
db_score = davies_bouldin_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
ch_score = calinski_harabasz_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])

print("\nInternal Validation Metrics:")
print("Silhouette Score:", round(silhouette, 4))
print("Davies-Bouldin Score:", round(db_score, 4))
print("Calinski-Harabasz Score:", round(ch_score, 4))

# Step 11: Biomarker discovery (differential features across clusters)
X_biomarker = result_df.drop('Cluster', axis=1).values
y_clusters = result_df['Cluster'].values

anova_pvals = []
kruskal_pvals = []

for i in range(X_biomarker.shape[1]):
    groups = [X_biomarker[y_clusters == c, i] for c in np.unique(y_clusters)]

    # ANOVA
    try:
        _, p_anova = f_oneway(*groups)
    except:
        p_anova = 1.0
    anova_pvals.append(p_anova)

    # Kruskal-Wallis
    try:
        _, p_kruskal = kruskal(*groups)
    except:
        p_kruskal = 1.0
    kruskal_pvals.append(p_kruskal)

biomarker_df = pd.DataFrame({
    'Feature': df.columns,  # use original biological names here
    'ANOVA_pval': anova_pvals,
    'Kruskal_pval': kruskal_pvals
})

biomarker_df.sort_values('Kruskal_pval', inplace=True)

print("\nTop Biomarker Candidates (based on Kruskal-Wallis test):")
print(biomarker_df.head(20))

# Step 12: Print only top biomarker names
top_n = 20
top_biomarkers = biomarker_df.head(top_n)['Feature'].values

print("\nTop Biomarker Feature Names:")
for name in top_biomarkers:
    print(name)

# Step 13: Heatmap of top biomarkers across samples sorted by cluster
top_features = biomarker_df.head(top_n)['Feature'].values
heatmap_data = result_df[top_features]
heatmap_data['Cluster'] = result_df['Cluster'].values
heatmap_data_sorted = heatmap_data.sort_values('Cluster')
cluster_labels = heatmap_data_sorted['Cluster']
heatmap_data_sorted = heatmap_data_sorted.drop('Cluster', axis=1)

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data_sorted.T, cmap='viridis', cbar=True, xticklabels=False)
plt.title("Heatmap of Top Biomarker Features Across Clusters")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load data
df = pd.read_csv('/content/HNSC_mrna (2).csv', sep='\t')  # Rows: samples, Columns: features (no true labels)
X = df.values

# Step 2: Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
# Step 4: Choose number of clusters
chosen_k = 2  # adjust based on elbow plot

# Step 5: Initial clustering to get pseudo-labels
initial_kmeans = KMeans(n_clusters=chosen_k, random_state=42)
pseudo_labels = initial_kmeans.fit_predict(X_scaled)

# Step 6: Apply SMOTE on pseudo-labels
smote = SMOTE(random_state=42)
X_smote, pseudo_labels_smote = smote.fit_resample(X_scaled, pseudo_labels)

# Step 7: Final clustering on SMOTE data
kmeans_final = KMeans(n_clusters=chosen_k, random_state=42)
final_clusters = kmeans_final.fit_predict(X_smote)

# Step 8: Create DataFrame with biological feature names and cluster labels
result_df = pd.DataFrame(X_smote, columns=df.columns)  # preserve real feature names here
result_df['Cluster'] = final_clusters

print(result_df.head())
print("\nCluster Counts:\n", result_df['Cluster'].value_counts())

# Step 9: t-SNE visualization of clusters
tsne = TSNE(n_components=2, random_state=42)
X_embedded = tsne.fit_transform(result_df.drop('Cluster', axis=1))

plt.figure(figsize=(8, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=result_df['Cluster'], cmap='viridis', s=10)
plt.title("t-SNE Visualization of Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label='Cluster')
plt.show()

# Step 10: Internal cluster validation scores
silhouette = silhouette_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
db_score = davies_bouldin_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
ch_score = calinski_harabasz_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])

print("\nInternal Validation Metrics:")
print("Silhouette Score:", round(silhouette, 4))
print("Davies-Bouldin Score:", round(db_score, 4))
print("Calinski-Harabasz Score:", round(ch_score, 4))

# Step 11: Biomarker discovery (differential features across clusters)
X_biomarker = result_df.drop('Cluster', axis=1).values
y_clusters = result_df['Cluster'].values

anova_pvals = []
kruskal_pvals = []

for i in range(X_biomarker.shape[1]):
    groups = [X_biomarker[y_clusters == c, i] for c in np.unique(y_clusters)]

    # ANOVA
    try:
        _, p_anova = f_oneway(*groups)
    except:
        p_anova = 1.0
    anova_pvals.append(p_anova)

    # Kruskal-Wallis
    try:
        _, p_kruskal = kruskal(*groups)
    except:
        p_kruskal = 1.0
    kruskal_pvals.append(p_kruskal)

biomarker_df = pd.DataFrame({
    'Feature': df.columns,  # use original biological names here
    'ANOVA_pval': anova_pvals,
    'Kruskal_pval': kruskal_pvals
})

biomarker_df.sort_values('Kruskal_pval', inplace=True)

print("\nTop Biomarker Candidates (based on Kruskal-Wallis test):")
print(biomarker_df.head(20))

# Step 12: Print only top biomarker names
top_n = 20
top_biomarkers = biomarker_df.head(top_n)['Feature'].values

print("\nTop Biomarker Feature Names:")
for name in top_biomarkers:
    print(name)

# Step 13: Heatmap of top biomarkers across samples sorted by cluster
top_features = biomarker_df.head(top_n)['Feature'].values
heatmap_data = result_df[top_features]
heatmap_data['Cluster'] = result_df['Cluster'].values
heatmap_data_sorted = heatmap_data.sort_values('Cluster')
cluster_labels = heatmap_data_sorted['Cluster']
heatmap_data_sorted = heatmap_data_sorted.drop('Cluster', axis=1)

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data_sorted.T, cmap='viridis', cbar=True, xticklabels=False)
plt.title("Heatmap of Top Biomarker Features Across Clusters")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

miRNA


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GaussianNoise, BatchNormalization
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import math



# Load the data with the correct delimiter ('\t' for tab-separated values)
miRNA = pd.read_csv('/content/HNSC_mirna (1).csv', sep='\t')

# Verify the data
print(miRNA.head())

# Ensure only numeric columns are processed
miRNA_numeric = mRNA.select_dtypes(include=['float64', 'int64'])

scaler = StandardScaler()
miRNA_scaled = scaler.fit_transform(miRNA)

# Define the autoencoder model
def create_denoising_autoencoder(input_dim, regularization=1e-5, dropout_rate=0.2, noise_factor=0.3):
    input_layer = Input(shape=(input_dim,))

    # Add Gaussian Noise for denoising
    noisy_input = GaussianNoise(noise_factor)(input_layer)

    # Encoder
    encoded = Dense(512, activation='relu', kernel_regularizer=l2(regularization))(noisy_input)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    encoded = Dense(256, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    encoded = Dense(128, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    encoded = Dense(64, activation='relu', activity_regularizer=l1(1e-5))(encoded)  # Sparse representation

    # Decoder
    decoded = Dense(128, activation='relu', kernel_regularizer=l2(regularization))(encoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dropout(dropout_rate)(decoded)
    decoded = Dense(256, activation='relu', kernel_regularizer=l2(regularization))(decoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dropout(dropout_rate)(decoded)
    decoded = Dense(input_dim, activation='linear')(decoded)

    autoencoder = Model(input_layer, decoded)
    return autoencoder

# Define the autoencoder model
input_dim = miRNA_scaled.shape[1]
autoencoder_miRNA = create_denoising_autoencoder(input_dim)

# Compile the autoencoder model with MSLE loss
autoencoder_miRNA.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredLogarithmicError())

# Define the cosine annealing learning rate scheduler
def cosine_annealing(epoch, lr, initial_lr=0.0001, min_lr=1e-5, T=2000):
    """Cosine annealing learning rate schedule."""
    cos_inner = (math.pi * (epoch % T)) / T
    return min_lr + (initial_lr - min_lr) * (1 + math.cos(cos_inner)) / 2

lr_schedule = LearningRateScheduler(lambda epoch, lr: cosine_annealing(epoch, lr, T=2000))

# Train the autoencoder on the full dataset
history_miRNA = autoencoder_miRNA.fit(
    miRNA_scaled,
    miRNA_scaled,
    epochs=2000,
    batch_size=128,
    callbacks=[lr_schedule]
)

# Print the final training loss
train_loss_miRNA = history_miRNA.history['loss'][-1]  # Get the last training loss
print(f'Training loss for miRNA dataset: {train_loss_miRNA}')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Reconstruct the miRNA data using the trained autoencoder
miRNA_reconstructed = autoencoder_miRNA.predict(miRNA_scaled)

# Convert the reconstructed data back to the original scale using the scaler
#miRNA_reconstructed_original = scaler_miRNA.inverse_transform(miRNA_reconstructed)

# Calculate the reconstruction errors
reconstruction_errors_miRNA = np.abs(miRNA_scaled - miRNA_reconstructed)

def plot_combined_heatmaps(original, reconstructed, errors, title):
    # Create a combined heatmap for original data, reconstructed data, and errors
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))

    sns.heatmap(original[:50], ax=axes[0], cmap='viridis', cbar=True)
    axes[0].set_title('Original miRNA Data')

    sns.heatmap(reconstructed[:50], ax=axes[1], cmap='viridis', cbar=True)
    axes[1].set_title('Reconstructed miRNA Data')

    sns.heatmap(errors[:50], ax=axes[2], cmap='Reds', cbar=True)
    axes[2].set_title('Reconstruction Error')

    plt.suptitle(title)
    plt.show()

# Plot the heatmaps for the miRNA dataset
plot_combined_heatmaps(miRNA_scaled, miRNA_reconstructed, reconstruction_errors_miRNA, 'miRNA Data Reconstruction Heatmaps')


In [ ]:
import matplotlib.pyplot as plt

def plot_loss_miRNA(history_miRNA):
    # Plot training loss
    plt.plot(history_miRNA.history['loss'], label='Training Loss')

    plt.title('miRNA Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

# Assuming history_miRNA is the variable holding the training history for miRNA
plot_loss_miRNA(history_miRNA)


In [ ]:
# Reconstruct the miRNA data using the trained autoencoder
X_test_reconstructed_miRNA = autoencoder_miRNA.predict(miRNA_scaled)

# Save reconstructed scaled data
reconstructed_miRNA_df_scaled = pd.DataFrame(X_test_reconstructed_miRNA, columns=miRNA.columns)
reconstructed_miRNA_df_scaled.to_csv('HNSC_mirna_reconstructed.csv', index=False)

# Optionally inverse-transform and save
X_test_reconstructed_original_miRNA = scaler.inverse_transform(X_test_reconstructed_miRNA)
reconstructed_miRNA_df_original = pd.DataFrame(X_test_reconstructed_original_miRNA, columns=miRNA.columns)
reconstructed_miRNA_df_original.to_csv('HNSC_mirna_original.csv', index=False)

print("Reconstructed miRNA datasets saved to 'reconstructed_miRNA_scaled.csv' and 'reconstructed_miRNA_original.csv'.")


In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load data
df = pd.read_csv('HNSC_mirna_original.csv')  # Rows: samples, Columns: features (no true labels)
X = df.values

# Step 2: Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
# Step 4: Choose number of clusters
chosen_k = 2  # adjust based on elbow plot

# Step 5: Initial clustering to get pseudo-labels
initial_kmeans = KMeans(n_clusters=chosen_k, random_state=42)
pseudo_labels = initial_kmeans.fit_predict(X_scaled)

# Step 6: Apply SMOTE on pseudo-labels
smote = SMOTE(random_state=42)
X_smote, pseudo_labels_smote = smote.fit_resample(X_scaled, pseudo_labels)

# Step 7: Final clustering on SMOTE data
kmeans_final = KMeans(n_clusters=chosen_k, random_state=42)
final_clusters = kmeans_final.fit_predict(X_smote)

# Step 8: Create DataFrame with biological feature names and cluster labels
result_df = pd.DataFrame(X_smote, columns=df.columns)  # preserve real feature names here
result_df['Cluster'] = final_clusters

print(result_df.head())
print("\nCluster Counts:\n", result_df['Cluster'].value_counts())

# Step 9: t-SNE visualization of clusters
tsne = TSNE(n_components=2, random_state=42)
X_embedded = tsne.fit_transform(result_df.drop('Cluster', axis=1))

plt.figure(figsize=(8, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=result_df['Cluster'], cmap='viridis', s=10)
plt.title("t-SNE Visualization of Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label='Cluster')
plt.show()

# Step 10: Internal cluster validation scores
silhouette = silhouette_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
db_score = davies_bouldin_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
ch_score = calinski_harabasz_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])

print("\nInternal Validation Metrics:")
print("Silhouette Score:", round(silhouette, 4))
print("Davies-Bouldin Score:", round(db_score, 4))
print("Calinski-Harabasz Score:", round(ch_score, 4))

# Step 11: Biomarker discovery (differential features across clusters)
X_biomarker = result_df.drop('Cluster', axis=1).values
y_clusters = result_df['Cluster'].values

anova_pvals = []
kruskal_pvals = []

for i in range(X_biomarker.shape[1]):
    groups = [X_biomarker[y_clusters == c, i] for c in np.unique(y_clusters)]

    # ANOVA
    try:
        _, p_anova = f_oneway(*groups)
    except:
        p_anova = 1.0
    anova_pvals.append(p_anova)

    # Kruskal-Wallis
    try:
        _, p_kruskal = kruskal(*groups)
    except:
        p_kruskal = 1.0
    kruskal_pvals.append(p_kruskal)

biomarker_df = pd.DataFrame({
    'Feature': df.columns,  # use original biological names here
    'ANOVA_pval': anova_pvals,
    'Kruskal_pval': kruskal_pvals
})

biomarker_df.sort_values('Kruskal_pval', inplace=True)

print("\nTop Biomarker Candidates (based on Kruskal-Wallis test):")
print(biomarker_df.head(20))

# Step 12: Print only top biomarker names
top_n = 20
top_biomarkers = biomarker_df.head(top_n)['Feature'].values

print("\nTop Biomarker Feature Names:")
for name in top_biomarkers:
    print(name)

# Step 13: Heatmap of top biomarkers across samples sorted by cluster
top_features = biomarker_df.head(top_n)['Feature'].values
heatmap_data = result_df[top_features]
heatmap_data['Cluster'] = result_df['Cluster'].values
heatmap_data_sorted = heatmap_data.sort_values('Cluster')
cluster_labels = heatmap_data_sorted['Cluster']
heatmap_data_sorted = heatmap_data_sorted.drop('Cluster', axis=1)

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data_sorted.T, cmap='viridis', cbar=True, xticklabels=False)
plt.title("Heatmap of Top Biomarker Features Across Clusters")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load data
df = pd.read_csv('/content/HNSC_mirna (1).csv', sep='\t')  # Rows: samples, Columns: features (no true labels)
X = df.values

# Step 2: Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
# Step 4: Choose number of clusters
chosen_k = 2  # adjust based on elbow plot

# Step 5: Initial clustering to get pseudo-labels
initial_kmeans = KMeans(n_clusters=chosen_k, random_state=42)
pseudo_labels = initial_kmeans.fit_predict(X_scaled)

# Step 6: Apply SMOTE on pseudo-labels
smote = SMOTE(random_state=42)
X_smote, pseudo_labels_smote = smote.fit_resample(X_scaled, pseudo_labels)

# Step 7: Final clustering on SMOTE data
kmeans_final = KMeans(n_clusters=chosen_k, random_state=42)
final_clusters = kmeans_final.fit_predict(X_smote)

# Step 8: Create DataFrame with biological feature names and cluster labels
result_df = pd.DataFrame(X_smote, columns=df.columns)  # preserve real feature names here
result_df['Cluster'] = final_clusters

print(result_df.head())
print("\nCluster Counts:\n", result_df['Cluster'].value_counts())

# Step 9: t-SNE visualization of clusters
tsne = TSNE(n_components=2, random_state=42)
X_embedded = tsne.fit_transform(result_df.drop('Cluster', axis=1))

plt.figure(figsize=(8, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=result_df['Cluster'], cmap='viridis', s=10)
plt.title("t-SNE Visualization of Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label='Cluster')
plt.show()

# Step 10: Internal cluster validation scores
silhouette = silhouette_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
db_score = davies_bouldin_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
ch_score = calinski_harabasz_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])

print("\nInternal Validation Metrics:")
print("Silhouette Score:", round(silhouette, 4))
print("Davies-Bouldin Score:", round(db_score, 4))
print("Calinski-Harabasz Score:", round(ch_score, 4))

# Step 11: Biomarker discovery (differential features across clusters)
X_biomarker = result_df.drop('Cluster', axis=1).values
y_clusters = result_df['Cluster'].values

anova_pvals = []
kruskal_pvals = []

for i in range(X_biomarker.shape[1]):
    groups = [X_biomarker[y_clusters == c, i] for c in np.unique(y_clusters)]

    # ANOVA
    try:
        _, p_anova = f_oneway(*groups)
    except:
        p_anova = 1.0
    anova_pvals.append(p_anova)

    # Kruskal-Wallis
    try:
        _, p_kruskal = kruskal(*groups)
    except:
        p_kruskal = 1.0
    kruskal_pvals.append(p_kruskal)

biomarker_df = pd.DataFrame({
    'Feature': df.columns,  # use original biological names here
    'ANOVA_pval': anova_pvals,
    'Kruskal_pval': kruskal_pvals
})

biomarker_df.sort_values('Kruskal_pval', inplace=True)

print("\nTop Biomarker Candidates (based on Kruskal-Wallis test):")
print(biomarker_df.head(20))

# Step 12: Print only top biomarker names
top_n = 20
top_biomarkers = biomarker_df.head(top_n)['Feature'].values

print("\nTop Biomarker Feature Names:")
for name in top_biomarkers:
    print(name)

# Step 13: Heatmap of top biomarkers across samples sorted by cluster
top_features = biomarker_df.head(top_n)['Feature'].values
heatmap_data = result_df[top_features]
heatmap_data['Cluster'] = result_df['Cluster'].values
heatmap_data_sorted = heatmap_data.sort_values('Cluster')
cluster_labels = heatmap_data_sorted['Cluster']
heatmap_data_sorted = heatmap_data_sorted.drop('Cluster', axis=1)

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data_sorted.T, cmap='viridis', cbar=True, xticklabels=False)
plt.title("Heatmap of Top Biomarker Features Across Clusters")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

Multiomics

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
reconstructed_df_original = pd.read_csv('/content/HNSC_Reconstruction_original.csv')
# Concatenate the reconstructed datasets
concatenated_reconstruction = pd.concat(
    [reconstructed_df_original, reconstructed_mRNA_df_original, reconstructed_miRNA_df_original],
    axis=1,
    #keys=["Meth", "mRNA", "miRNA"]
)

# Save the concatenated dataset to a CSV file
concatenated_reconstruction.to_csv("HNSC_reconstructed_multiomics.csv", index=False)
print("Concatenated reconstruction dataset saved to 'HNSC_multi_omics.csv.csv'.")

# Visualize the heatmap of the concatenated data (taking the first 2503 samples)
plt.figure(figsize=(20, 20))
sns.heatmap(concatenated_reconstruction.iloc[:2503].T, cmap="viridis")  # Transpose to have features on the y-axis
plt.title("Heatmap of Concatenated Reconstruction Data")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.show()


In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load data
df = pd.read_csv('HNSC_reconstructed_multiomics.csv')  # Rows: samples, Columns: features (no true labels)
X = df.values

# Step 2: Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
# Step 4: Choose number of clusters
chosen_k = 2  # adjust based on elbow plot

# Step 5: Initial clustering to get pseudo-labels
initial_kmeans = KMeans(n_clusters=chosen_k, random_state=42)
pseudo_labels = initial_kmeans.fit_predict(X_scaled)

# Step 6: Apply SMOTE on pseudo-labels
smote = SMOTE(random_state=42)
X_smote, pseudo_labels_smote = smote.fit_resample(X_scaled, pseudo_labels)

# Step 7: Final clustering on SMOTE data
kmeans_final = KMeans(n_clusters=chosen_k, random_state=42)
final_clusters = kmeans_final.fit_predict(X_smote)

# Step 8: Create DataFrame with biological feature names and cluster labels
result_df = pd.DataFrame(X_smote, columns=df.columns)  # preserve real feature names here
result_df['Cluster'] = final_clusters

print(result_df.head())
print("\nCluster Counts:\n", result_df['Cluster'].value_counts())

# Step 9: t-SNE visualization of clusters
tsne = TSNE(n_components=2, random_state=42)
X_embedded = tsne.fit_transform(result_df.drop('Cluster', axis=1))

plt.figure(figsize=(8, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=result_df['Cluster'], cmap='viridis', s=10)
plt.title("t-SNE Visualization of Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label='Cluster')
plt.show()

# Step 10: Internal cluster validation scores
silhouette = silhouette_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
db_score = davies_bouldin_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
ch_score = calinski_harabasz_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])

print("\nInternal Validation Metrics:")
print("Silhouette Score:", round(silhouette, 4))
print("Davies-Bouldin Score:", round(db_score, 4))
print("Calinski-Harabasz Score:", round(ch_score, 4))

# Step 11: Biomarker discovery (differential features across clusters)
X_biomarker = result_df.drop('Cluster', axis=1).values
y_clusters = result_df['Cluster'].values

anova_pvals = []
kruskal_pvals = []

for i in range(X_biomarker.shape[1]):
    groups = [X_biomarker[y_clusters == c, i] for c in np.unique(y_clusters)]

    # ANOVA
    try:
        _, p_anova = f_oneway(*groups)
    except:
        p_anova = 1.0
    anova_pvals.append(p_anova)

    # Kruskal-Wallis
    try:
        _, p_kruskal = kruskal(*groups)
    except:
        p_kruskal = 1.0
    kruskal_pvals.append(p_kruskal)

biomarker_df = pd.DataFrame({
    'Feature': df.columns,  # use original biological names here
    'ANOVA_pval': anova_pvals,
    'Kruskal_pval': kruskal_pvals
})

biomarker_df.sort_values('Kruskal_pval', inplace=True)

print("\nTop Biomarker Candidates (based on Kruskal-Wallis test):")
print(biomarker_df.head(20))

# Step 12: Print only top biomarker names
top_n = 20
top_biomarkers = biomarker_df.head(top_n)['Feature'].values

print("\nTop Biomarker Feature Names:")
for name in top_biomarkers:
    print(name)

# Step 13: Heatmap of top biomarkers across samples sorted by cluster
top_features = biomarker_df.head(top_n)['Feature'].values
heatmap_data = result_df[top_features]
heatmap_data['Cluster'] = result_df['Cluster'].values
heatmap_data_sorted = heatmap_data.sort_values('Cluster')
cluster_labels = heatmap_data_sorted['Cluster']
heatmap_data_sorted = heatmap_data_sorted.drop('Cluster', axis=1)

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data_sorted.T, cmap='viridis', cbar=True, xticklabels=False)
plt.title("Heatmap of Top Biomarker Features Across Clusters")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
meth = pd.read_csv('/content/HNSC_methy (1).csv', sep='\t')
mRNA = pd.read_csv('/content/HNSC_mrna (2).csv', sep='\t')
miRNA = pd.read_csv('/content/HNSC_mirna (1).csv', sep='\t')
# Concatenate the reconstructed datasets
concatenated_reconstruction = pd.concat(
    [meth, mRNA, miRNA],
    axis=1,
    #keys=["Meth", "mRNA", "miRNA"]
)

# Save the concatenated dataset to a CSV file
concatenated_reconstruction.to_csv("HNSC_MO.csv", index=False)
print("Concatenated reconstruction dataset saved to 'HNSC_multi_omics.csv.csv'.")

# Visualize the heatmap of the concatenated data (taking the first 2503 samples)
plt.figure(figsize=(20, 20))
sns.heatmap(concatenated_reconstruction.iloc[:2503].T, cmap="viridis")  # Transpose to have features on the y-axis
plt.title("Heatmap of Concatenated Reconstruction Data")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.show()


In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load data
df = pd.read_csv('/content/HNSC_MO.csv')  # Rows: samples, Columns: features (no true labels)
X = df.values

# Step 2: Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Elbow method
inertia = []
k_range = range(2, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()


In [ ]:
# Step 4: Choose number of clusters
chosen_k = 2  # adjust based on elbow plot

# Step 5: Initial clustering to get pseudo-labels
initial_kmeans = KMeans(n_clusters=chosen_k, random_state=42)
pseudo_labels = initial_kmeans.fit_predict(X_scaled)

# Step 6: Apply SMOTE on pseudo-labels
smote = SMOTE(random_state=42)
X_smote, pseudo_labels_smote = smote.fit_resample(X_scaled, pseudo_labels)

# Step 7: Final clustering on SMOTE data
kmeans_final = KMeans(n_clusters=chosen_k, random_state=42)
final_clusters = kmeans_final.fit_predict(X_smote)

# Step 8: Create DataFrame with biological feature names and cluster labels
result_df = pd.DataFrame(X_smote, columns=df.columns)  # preserve real feature names here
result_df['Cluster'] = final_clusters

print(result_df.head())
print("\nCluster Counts:\n", result_df['Cluster'].value_counts())

# Step 9: t-SNE visualization of clusters
tsne = TSNE(n_components=2, random_state=42)
X_embedded = tsne.fit_transform(result_df.drop('Cluster', axis=1))

plt.figure(figsize=(8, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=result_df['Cluster'], cmap='viridis', s=10)
plt.title("t-SNE Visualization of Clusters")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(label='Cluster')
plt.show()

# Step 10: Internal cluster validation scores
silhouette = silhouette_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
db_score = davies_bouldin_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])
ch_score = calinski_harabasz_score(result_df.drop('Cluster', axis=1), result_df['Cluster'])

print("\nInternal Validation Metrics:")
print("Silhouette Score:", round(silhouette, 4))
print("Davies-Bouldin Score:", round(db_score, 4))
print("Calinski-Harabasz Score:", round(ch_score, 4))

# Step 11: Biomarker discovery (differential features across clusters)
X_biomarker = result_df.drop('Cluster', axis=1).values
y_clusters = result_df['Cluster'].values

anova_pvals = []
kruskal_pvals = []

for i in range(X_biomarker.shape[1]):
    groups = [X_biomarker[y_clusters == c, i] for c in np.unique(y_clusters)]

    # ANOVA
    try:
        _, p_anova = f_oneway(*groups)
    except:
        p_anova = 1.0
    anova_pvals.append(p_anova)

    # Kruskal-Wallis
    try:
        _, p_kruskal = kruskal(*groups)
    except:
        p_kruskal = 1.0
    kruskal_pvals.append(p_kruskal)

biomarker_df = pd.DataFrame({
    'Feature': df.columns,  # use original biological names here
    'ANOVA_pval': anova_pvals,
    'Kruskal_pval': kruskal_pvals
})

biomarker_df.sort_values('Kruskal_pval', inplace=True)

print("\nTop Biomarker Candidates (based on Kruskal-Wallis test):")
print(biomarker_df.head(20))

# Step 12: Print only top biomarker names
top_n = 20
top_biomarkers = biomarker_df.head(top_n)['Feature'].values

print("\nTop Biomarker Feature Names:")
for name in top_biomarkers:
    print(name)

# Step 13: Heatmap of top biomarkers across samples sorted by cluster
top_features = biomarker_df.head(top_n)['Feature'].values
heatmap_data = result_df[top_features]
heatmap_data['Cluster'] = result_df['Cluster'].values
heatmap_data_sorted = heatmap_data.sort_values('Cluster')
cluster_labels = heatmap_data_sorted['Cluster']
heatmap_data_sorted = heatmap_data_sorted.drop('Cluster', axis=1)

plt.figure(figsize=(14, 8))
sns.heatmap(heatmap_data_sorted.T, cmap='viridis', cbar=True, xticklabels=False)
plt.title("Heatmap of Top Biomarker Features Across Clusters")
plt.xlabel("Samples")
plt.ylabel("Features")
plt.tight_layout()
plt.show()